## Setup

#### Load relevant Python libaries.


In [ ]:
import json
import os
from pathlib import Path

from risk_atlas_nexus.blocks.inference import OllamaInferenceEngine, RITSInferenceEngine
from risk_atlas_nexus.blocks.inference.params import (
    InferenceEngineCredentials,
    OllamaInferenceEngineParams,
    RITSInferenceEngineParams,
)
from risk_atlas_nexus.data import get_templates_path
from risk_atlas_nexus.library import RiskAtlasNexus

##### Risk Atlas Nexus uses Large Language Models (LLMs) to infer risks dimensions. Therefore requires access to LLMs to inference or call the model.

**Available Inference Engines**: WML, Ollama, vLLM, RITS. Please follow the [Inference APIs](https://github.com/IBM/risk-atlas-nexus?tab=readme-ov-file#install-for-inference-apis) guide before going ahead.

_Note:_ RITS is intended solely for internal IBM use and requires TUNNELALL VPN for access.


In [ ]:
# inference_engine = RITSInferenceEngine(
#     model_name_or_path="meta-llama/llama-3-3-70b-instruct",
#     credentials={
#         "api_key": "RITS_API_KEY",
#         "api_url": "https://inference-3scale-apicast-production.apps.rits.fmaas.res.ibm.com",
#     },
#     parameters=RITSInferenceEngineParams(max_tokens=1000, temperature=0),
# )

inference_engine = OllamaInferenceEngine(
    model_name_or_path="granite3.2:8b",
    credentials=InferenceEngineCredentials(api_url="OLLAMA_API_URL"),
    parameters=OllamaInferenceEngineParams(
        num_predict=1000, temperature=0, repeat_penalty=1, num_ctx=8192
    ),
)

[2025-05-19 19:18:29:923] - INFO - RiskAtlasNexus - OLLAMA inference engine will execute requests on the server at localhost:11434.
[2025-05-19 19:18:29:968] - INFO - RiskAtlasNexus - Created OLLAMA inference engine.


##### Create an instance of RiskAtlasNexus

_Note: (Optional)_ You can specify your own directory in `RiskAtlasNexus(base_dir=<PATH>)` to utilize custom AI ontologies. If left blank, the system will use the provided AI ontologies.


In [3]:
risk_atlas_nexus = RiskAtlasNexus()

[2025-05-19 19:18:30:137] - INFO - RiskAtlasNexus - Created RiskAtlasNexus instance. Base_dir: None


### UseIntents


In [4]:
userIntent1 = "Generate personalized, relevant responses, recommendations, and summaries of claims for customers to support agents to enhance their interactions with customers."
print("userIntent1: ", userIntent1)
print()
userIntent2 = "In a medical chatbot, generative AI can be employed to create a triage system that assesses patients' symptoms and provides immediate, contextually relevant advice based on their medical history and current condition. The chatbot can analyze the patient's input, identify potential medical issues, and offer tailored recommendations or insights to the patient or healthcare provider. This can help streamline the triage process, ensuring that patients receive the appropriate. level of care and attention, and ultimately improving patient outcomes."
print("userIntent2: ", userIntent2)

userIntent1:  Generate personalized, relevant responses, recommendations, and summaries of claims for customers to support agents to enhance their interactions with customers.

userIntent2:  In a medical chatbot, generative AI can be employed to create a triage system that assesses patients' symptoms and provides immediate, contextually relevant advice based on their medical history and current condition. The chatbot can analyze the patient's input, identify potential medical issues, and offer tailored recommendations or insights to the patient or healthcare provider. This can help streamline the triage process, ensuring that patients receive the appropriate. level of care and attention, and ultimately improving patient outcomes.


#### Chain of Thought Data


In [3]:
CoT = json.loads(
    Path(os.path.join(get_templates_path(), "cot_examples.json")).read_text()
)
CoT[0]

{'question': 'What domain does your use request fall under? Customer service/support, Technical, Information retrieval, Strategy, Code/software engineering, Communications, IT/business automation, Writing assistant, Financial, Talent and Organization including HR, Product, Marketing, Cybersecurity, Healthcare, User Research, Sales, Risk and Compliance, Design, Other',
 'examples': [{'intent': 'Optimize supply chain management in Investment banks',
   'answer': 'Strategy',
   'explanation': 'Since the task is involved in improving the processes to ensure better performance. It is not finance since the task is on supply chain optimization and not on financial aspects even though the application domain is banks.'},
  {'intent': 'Ability to create dialog flows and integrations from natural language instructions.',
   'answer': 'Customer service/support',
   'explanation': 'Since the task relates to human conversations or generating human converstations or support.'},
  {'intent': 'Check if

#### Invoking Risk Severity Service on userIntent1 and userIntent2


In [6]:
response = risk_atlas_nexus.categorize_risk_severity(
    usecases=[userIntent1, userIntent2], cot_data=CoT, inference_engine=inference_engine
)

[2025-05-19 19:18:44:99] - INFO - RiskAtlasNexus - Domain: ['Customer service/support', 'Healthcare']
[2025-05-19 19:19:04:667] - INFO - RiskAtlasNexus - AI Tasks: [['Text-to-Image', 'Text-to-Speech', 'Text Generation', 'Summarization', 'Text Classification', 'Text-to-Video', 'Text-to-3D', 'Image-to-3D'], ['Text-to-Image', 'Text-to-Speech', 'Text Classification', 'Summarization', 'Question Answering', 'Feature Extraction', 'Image Classification', 'Image Segmentation', 'Image-to-Image', 'Image Feature Extraction', 'Image-Text-to-Text', 'Image-to-Text', 'Keypoint Detection', 'Mask Generation', 'Object Detection', 'Text Generation', 'Token Classification', 'Translation', 'Unconditional Image Generation', 'Video-Text-to-Text', 'Visual Question Answering', 'Zero-Shot Classification', 'Zero-Shot Image Classification']]
[2025-05-19 19:19:16:894] - INFO - RiskAtlasNexus - AI User: [['Customer Support Agents'], ['Patients and Healthcare Providers']]
[2025-05-19 19:19:30:663] - INFO - RiskAtlasN

#### Risk Severity


In [7]:
response[0]

{'Description': 'The AI system intended to be used by customer support agents to generate personalized, relevant responses, recommendations, and summaries of claims for customers, enhancing their interactions with customers.',
 'Classification': ['Minimal Risk'],
 'Relevant Text from the EU AI Act': {},
 'Reasoning': 'The AI system described does not fall under any of the high-risk categories outlined in the EU AI Act. It does not involve biometric identification, critical infrastructure management, or decision-making in areas such as employment, public services, or law enforcement. Instead, it focuses on customer service tasks like text generation, summarization, and text classification, which are not considered high-risk activities according to the Act.'}

In [8]:
response[1]

{'Description': "The AI system intended to be used in a medical chatbot that assesses patients' symptoms and provides immediate, contextually relevant advice based on their medical history and current condition.",
 'Classification': ['High Risk'],
 'Relevant Text from the EU AI Act': {'Section': 'Annex III – paragraph 1 – point 1 – point a',
  'Amendment': 'Amendment 712',
  'Text': 'AI systems intended to be used for biometric identification of natural persons, with the exception of those mentioned in Article 5;',
  'Reason': 'The AI system involves biometric identification of patients, which falls under this category.'},
 'Reasoning': "The AI system described involves biometric identification of patients, which is classified as a high-risk activity under the EU AI Act. The amendment 712 specifically mentions AI systems intended for biometric identification, excluding those mentioned in Article 5. This aligns with the AI system's purpose of assessing patients' symptoms and providing a